In [ ]:
!sudo mkdir files
!sudo mkdir files/S1
!sudo mkdir files/QC_v2
!sudo mkdir files/QC_v2_shrunk
!gsutil -m rsync -r gs://cnn_chips/S1 files/S1
#!gsutil -m rsync -r gs://refined_cnn_chips/QC_v2_shrunk files/QC_v2_shrunk
!gsutil -m rsync -r gs://cnn_chips/QC_v2 files/QC_v2

mkdir: cannot create directory ‘files’: File exists
mkdir: cannot create directory ‘files/S1’: File exists
mkdir: cannot create directory ‘files/QC_v2’: File exists
mkdir: cannot create directory ‘files/QC_v2_shrunk’: File exists
Building synchronization state...
Starting synchronization...
Building synchronization state...
Starting synchronization...


In [ ]:
from google.colab import auth
auth.authenticate_user()

!curl https://sdk.cloud.google.com | bash

!gcloud init

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   443  100   443    0     0  31642      0 --:--:-- --:--:-- --:--:-- 34076
######################################################################## 100.0%
Running install script from: /tmp/tmp.3apzS1z16E/install_google_cloud_sdk.bash
which curl
curl -# -f https://dl.google.com/dl/cloudsdk/channels/rapid/google-cloud-sdk.tar.gz
######################################################################## 100.0%

mkdir -p /root
"/root/google-cloud-sdk" already exists and may contain out of date files.
Remove /root/google-cloud-sdk or select a new installation directory, then run again.
Welcome! This command will take you through the configuration of gcloud.

Settings from your current configuration [default] are:
component_manager:
  disable_update_check: 'True'
compute:
  gce_metadata_read_timeout_sec: '0'
core:
  account: matthewjo

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  19787      0 --:--:-- --:--:-- --:--:-- 19787
OK
53 packages can be upgraded. Run 'apt list --upgradable' to see them.
gcsfuse is already the newest version (0.30.0).
0 upgraded, 0 newly installed, 0 to remove and 53 not upgraded.


In [ ]:
!cd /home
!pwd

/content


In [ ]:
!pip install rasterio

!gsutil cp gs://cnn_chips/flood_test_data.csv .
!gsutil cp gs://cnn_chips/flood_train_data.csv .
!gsutil cp gs://cnn_chips/flood_valid_data.csv .
!gsutil cp gs://cnn_chips/shrunk_flood_test_data.csv .

Copying gs://cnn_chips/flood_test_data.csv...
/ [1 files][  4.5 KiB/  4.5 KiB]                                                
Operation completed over 1 objects/4.5 KiB.                                      
Copying gs://cnn_chips/flood_train_data.csv...
/ [1 files][ 12.5 KiB/ 12.5 KiB]                                                
Operation completed over 1 objects/12.5 KiB.                                     
Copying gs://cnn_chips/flood_valid_data.csv...
/ [1 files][  4.4 KiB/  4.4 KiB]                                                
Operation completed over 1 objects/4.4 KiB.                                      
CommandException: No URLs matched: gs://cnn_chips/shrunk_flood_test_data.csv


In [ ]:
LR = 1e-3
EPOCHS = 1000
EPOCHS_PER_UPDATE = 1
RUNNAME = "1e3_flood_0"
BASEDIR = ''

MODEL_NAME = 'refiner'  # [unet, fcn, refiner]
CS_POINTS_CLUSTERING = None  # [None, low, high]
CS_POINTS_NOISE = None  # [None, low, high]

In [ ]:
# get crowd source points data path
def get_cs_points_path(base_dir, cluster_type, noise_type):
    try:
        cluster_type = cluster_type.lower()
        noise_type = noise_type.lower()
    except AttributeError:
        # either cluster_type or noise_type is None
        return None

    if not cluster_type in ['low', 'high']:
        print('FATAL: Invalid cluster type {}'.format(cluster_type))
        exit()

    if not noise_type in ['low', 'high']:
        print('FATAL: Invalid noise type {}'.format(noise_type))
        exit()

    cs_points_path = os.path.join(base_dir, 'QC_v2_cluster_{}_noise_{}.p'.format(cluster_type, noise_type))

    return cs_points_path

crowd_points_path = get_cs_points_path(BASEDIR, CS_POINTS_CLUSTERING, CS_POINTS_NOISE)


In [ ]:
import random
from PIL import Image

import torch
from torchvision import transforms
import torchvision.transforms.functional as F

class InMemoryDataset(torch.utils.data.Dataset):
  
  def __init__(self, data_list, preprocess_func):
    self.data_list = data_list
    self.preprocess_func = preprocess_func    
  
  def __getitem__(self, i):
    return self.preprocess_func(self.data_list[i])
  
  def __len__(self):
    return len(self.data_list)


def processAndAugment(data):
  output = {}
  try:
    (x,y) = data
    p = None
  except:
    x,y,p = data  # image, mask, point image
    p = Image.fromarray(p)
  im, label = x.copy(), y.copy()

  # convert to PIL for easier transforms
  im1 = Image.fromarray(im[0])
  im2 = Image.fromarray(im[1])
  label = Image.fromarray(label.squeeze())

  # Get params for random transforms
  i, j, h, w = transforms.RandomCrop.get_params(im1, (256, 256))
  
  im1 = F.crop(im1, i, j, h, w)
  im2 = F.crop(im2, i, j, h, w)
  label = F.crop(label, i, j, h, w)
  if not p is None:
    p = F.crop(p, i, j, h, w)

  if random.random() > 0.5:
    im1 = F.hflip(im1)
    im2 = F.hflip(im2)
    label = F.hflip(label)
    if not p is None:
      p = F.hflip(p)

  if random.random() > 0.5:
    im1 = F.vflip(im1)
    im2 = F.vflip(im2)
    label = F.vflip(label)
    if not p is None:
      p = F.vflip(p)

  norm = transforms.Normalize([0.6851, 0.5235], [0.0820, 0.1102])
  im = torch.stack([transforms.ToTensor()(im1).squeeze(), transforms.ToTensor()(im2).squeeze()])
  im = norm(im)
  label = transforms.ToTensor()(label).squeeze()
  if torch.sum(label.gt(.003) * label.lt(.004)):
    label *= 255
  label = label.round()

  output['image'] = im
  output['label'] = label

  if not p is None:
    p = F.to_tensor(p)
    output['point_img'] = p

  return output


def processTestIm(data):
  output = {}
  try:
    (x,y) = data
    p = None
  except:
    x,y,p = data  # image, mask, point image
    p = Image.fromarray(p)

  im, label = x.copy(), y.copy()

  norm = transforms.Normalize([0.6851, 0.5235], [0.0820, 0.1102])
  #label[0][0][0] = 255
  
  # convert to PIL for easier transforms
  im_c1 = Image.fromarray(im[0]).resize((512,512))
  im_c2 = Image.fromarray(im[1]).resize((512,512))
  label = Image.fromarray(label.squeeze()).resize((512,512))

  im_c1s = [F.crop(im_c1, 0, 0, 256, 256), F.crop(im_c1, 0, 256, 256, 256),
            F.crop(im_c1, 256, 0, 256, 256), F.crop(im_c1, 256, 256, 256, 256)]
  im_c2s = [F.crop(im_c2, 0, 0, 256, 256), F.crop(im_c2, 0, 256, 256, 256),
            F.crop(im_c2, 256, 0, 256, 256), F.crop(im_c2, 256, 256, 256, 256)]
  labels = [F.crop(label, 0, 0, 256, 256), F.crop(label, 0, 256, 256, 256),
            F.crop(label, 256, 0, 256, 256), F.crop(label, 256, 256, 256, 256)]
  if not p is None:
    p = [F.crop(p, 0, 0, 256, 256), F.crop(p, 0, 256, 256, 256),
         F.crop(p, 256, 0, 256, 256), F.crop(p, 256, 256, 256, 256)]
    p = [F.to_tensor(p_img) for p_img in p]
    p = torch.stack(p)

  ims = [torch.stack((transforms.ToTensor()(x).squeeze(),
                    transforms.ToTensor()(y).squeeze()))
                    for (x,y) in zip(im_c1s, im_c2s)]
  ims = [norm(im) for im in ims]
  ims = torch.stack(ims)
  labels = [(transforms.ToTensor()(label).squeeze()) for label in labels]
  labels = torch.stack(labels)
  if torch.sum(labels.gt(.003) * labels.lt(.004)):
    labels *= 255
  labels = labels.round()

  output['image'] = ims
  output['label'] = labels

  if not p is None:
    output['point_img'] = p
  return output

In [ ]:
import csv
import pickle
from PIL import Image
import rasterio
import numpy as np
import os


def getArr(fname):
  return rasterio.open('files/' + fname).read()


def download_perm_water_data_from_file(fname, crowd_points_path=None):
  if not crowd_points_path is None:
    try:
      crowd_points_dict = pickle.load(open(crowd_points_path, 'rb'))
    except IOError:
      print('Cant access crowd source points, using fake points.')
      crowd_points_dict = {}
  with open(fname) as f:
    data_fnames = [tuple(line) for line in csv.reader(f)]
  i = 0
  data = []
  for (x,y) in data_fnames:
    # print(x, y)
    arr_x, arr_y = getArr(x), getArr(y)
    if np.sum((arr_x != arr_x)) == 0:
      ignore = (arr_y == -1)
      ignore = ((np.uint8(ignore) * -1) * 256) + 1
      arr_y *= ignore
      if not crowd_points_path is None:
        # get image name
        img_name = os.path.splitext(os.path.split(x)[1])[0]
        try:
          point_image = crowd_points_dict[img_name]
        except KeyError:
          point_image = np.zeros((512, 512))
        data.append((arr_x, arr_y, point_image))
      else:
        data.append((arr_x, arr_y))
      i+=1
      print(i)
    else:
      print("skipping nan")
  return data

def download_perm_train_data():
  TRAINING_DATA_FILE = BASEDIR + 'flood_train_data.csv'
  return download_perm_water_data_from_file(TRAINING_DATA_FILE, crowd_points_path)

def download_perm_valid_data():
  VALID_DATA_FILE = BASEDIR + 'flood_valid_data.csv'
  return download_perm_water_data_from_file(VALID_DATA_FILE, crowd_points_path)

def download_perm_test_data():
  TEST_DATA_FILE = BASEDIR + 'flood_test_data.csv'
  return download_perm_water_data_from_file(TEST_DATA_FILE, crowd_points_path)

In [ ]:
from time import time

def getArrFlood(fname):
  return rasterio.open(fname).read()

def download_flood_water_data_from_list(l, crowd_points_path=None):
  if not crowd_points_path is None:
    try:
      crowd_points_dict = pickle.load(open(crowd_points_path, 'rb'))
    except IOError:
      print('Cant access crowd source points, using fake points.')
      crowd_points_dict = {}
  i= 0
  tot_nan = 0
  tot_good = 0
  flood_data = []
  for (im_fname, mask_fname) in l:
    print(im_fname)
    if not os.path.exists(os.path.join("files/", im_fname)):
      print(os.path.join("files/", im_fname))
      continue
    arr_x = np.nan_to_num(getArrFlood(os.path.join("files/", im_fname)))
    arr_y = getArrFlood(os.path.join("files/", mask_fname))
    ignore = (arr_y == -1)
    ignore = ((np.uint8(ignore) * -1) * 256) + 1
    # arr_y *= ignore
    arr_y = np.uint8(getArrFlood(os.path.join("files/", mask_fname)))
    if np.sum((arr_y != arr_y)) == 0:
      arr_x = np.clip(arr_x, -50, 1)
      arr_x = (arr_x + 50) / 51
      if i % 100 == 0:
        print(i)
        print(im_fname, mask_fname)
      i += 1
      if not crowd_points_path is None:
        # get image name
        img_name = os.path.splitext(os.path.split(im_fname)[1])[0]
        try:
          point_image = crowd_points_dict[img_name]
        except KeyError:
          point_image = np.zeros((512, 512))
        flood_data.append((arr_x, arr_y, point_image))
      else:
        flood_data.append((arr_x, arr_y))
    else:
      print("skipping nan")
  print(i)
  return flood_data

def load_flood_train_data():
  basedir = ""
  fname = "flood_train_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  return download_flood_water_data_from_list(fname, crowd_points_path)

def load_weak_flood_train_data():
  basedir = ""
  files = [(os.path.join("S1_NoQC", x[1]), os.path.join("NoQC", x[0])) for x in zip(sorted(os.listdir("files/NoQC")), sorted(os.listdir("files/S1_NoQC")))]
  files = [x for x in files if "Bolivia" not in x[0]]
  print(files[0:10])
  return download_flood_water_data_from_list(files, crowd_points_path)

def load_flood_test_perm_data():
  fname = "flood_test_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  name = [(t[0], t[1].replace("QC_v2", "Perm").replace("QC", "Perm")) for t in fname]
  return download_flood_water_data_from_list(fname, crowd_points_path)

def load_flood_valid_data():
  basedir = ""
  fname = "flood_valid_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  print(fname, "files!")
  return download_flood_water_data_from_list(fname, crowd_points_path)

def load_flood_test_data():
  basedir = ""
  fname = "flood_test_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  return download_flood_water_data_from_list(fname, crowd_points_path)

def load_flood_bolivia_test_data():
  basedir = ""
  fname = "flood_bolivia_data.csv"
  with open(fname) as f:
    fname = [tuple(line) for line in csv.reader(f)]
  return download_flood_water_data_from_list(fname, crowd_points_path)

In [ ]:
!ls
train_data = download_perm_train_data()
train_dataset = InMemoryDataset(train_data, processAndAugment)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=None,
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
train_iter = iter(train_loader)

adc.json     files		  flood_train_data.csv	sample_data
checkpoints  flood_test_data.csv  flood_valid_data.csv
1
2
3
4
5
skipping nan
6
7
8
9
10
11
12
13
14
skipping nan
15
skipping nan
16
17
18
19
20
21
22
skipping nan
23
24
25
26
skipping nan
27
28
29
skipping nan
30
31
32
33
34
skipping nan
35
36
37
38
39
40
skipping nan
41
42
43
44
skipping nan
45
46
47
48
49
50
51
52
53
54
skipping nan
55
56
57
58
59
60
61
62
63
skipping nan
64
65
66
skipping nan
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
skipping nan
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
skipping nan
136
137
138
139
140
141
142
143
144
145
146
147
148
skipping nan
149
150
151
152
skipping nan
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
19

In [ ]:
mkdir checkpoints

mkdir: cannot create directory ‘checkpoints’: File exists


In [ ]:
flood_test_perm_data = load_flood_test_perm_data()
flood_test_all_data = load_flood_test_data()

S1/Ghana_313799_S1.tif
0
S1/Ghana_313799_S1.tif QC_v2/Ghana_313799_QC.tif
S1/Ghana_1078550_S1.tif
S1/Ghana_97059_S1.tif
S1/Ghana_359826_S1.tif
S1/Ghana_319168_S1.tif
S1/Ghana_866994_S1.tif
S1/Ghana_406026_S1.tif
S1/Ghana_53713_S1.tif
S1/Ghana_83483_S1.tif
S1/Ghana_167233_S1.tif
S1/Ghana_141271_S1.tif
S1/India_900498_S1.tif
S1/India_591317_S1.tif
S1/India_747992_S1.tif
S1/India_79637_S1.tif
S1/India_952728_S1.tif
S1/India_828067_S1.tif
S1/India_570384_S1.tif
S1/India_44475_S1.tif
S1/India_80221_S1.tif
S1/India_1018327_S1.tif
S1/India_592446_S1.tif
S1/India_772630_S1.tif
S1/India_631692_S1.tif
S1/India_399883_S1.tif
S1/Mekong_333434_S1.tif
S1/Mekong_45934_S1.tif
S1/Mekong_1443339_S1.tif
S1/Mekong_382276_S1.tif
S1/Mekong_254910_S1.tif
S1/Mekong_424793_S1.tif
S1/Nigeria_417184_S1.tif
S1/Nigeria_225131_S1.tif
S1/Nigeria_812045_S1.tif
S1/Nigeria_22088_S1.tif
S1/Pakistan_849790_S1.tif
S1/Pakistan_664885_S1.tif
S1/Pakistan_694942_S1.tif
S1/Pakistan_70625_S1.tif
S1/Pakistan_528249_S1.tif
S1/Pak

In [ ]:
def ignore_perm_water(all_water, perm_water):
  w = all_water.copy()
  perm_water = (w == 1) * (perm_water == 1)
  w[perm_water] = 255
  return w

def ignore_flood_water(all_water, perm_water):
  w = all_water.copy()
  flood_water = (w == 1) * (perm_water == 0)
  w[flood_water] = 255
  return w

def get_flood_flood_test_data(all_water_data, perm_water_data):
  flood_flood_test_data = []
  for (all_water_d, perm_water_d) in zip(all_water_data, perm_water_data):
    # pwy = ignore_perm_water(awy, pwy)
    if np.sum(all_water_d[0] == 1) > 0 and np.sum(perm_water_d[0] == 1) == 0:
      continue
    flood_flood_test_data.append(all_water_d)
  return flood_flood_test_data
  

def get_perm_flood_test_data(all_water_data, perm_water_data):
  perm_flood_test_data = []
  for (all_water_d, perm_water_d) in zip(all_water_data, perm_water_data):
    # pwy = ignore_flood_water(awy, pwy)
    if np.sum(all_water_d[0] == 1) > 0 and np.sum(perm_water_d[0] == 1) == 0:
      continue
    perm_flood_test_data.append(all_water_d)
  return perm_flood_test_data

In [ ]:
flood_flood_test_data = get_flood_flood_test_data(flood_test_all_data, flood_test_perm_data)
perm_flood_test_data = get_perm_flood_test_data(flood_test_all_data, flood_test_perm_data)

In [ ]:
test_all_data = flood_test_all_data
test_all_dataset = InMemoryDataset(test_all_data, processTestIm)
test_all_loader = torch.utils.data.DataLoader(test_all_dataset, batch_size=1, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=lambda x: x[0],
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
test_all_iter = iter(test_all_loader)

test_flood_data = flood_flood_test_data
test_flood_dataset = InMemoryDataset(test_flood_data, processTestIm)
test_flood_loader = torch.utils.data.DataLoader(test_flood_dataset, batch_size=1, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=lambda x: x[0],
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
test_flood_iter = iter(test_flood_loader)

valid_data = load_flood_valid_data() #download_perm_valid_data()
valid_dataset = InMemoryDataset(valid_data, processTestIm)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=4, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=lambda x: (torch.cat([a[0] for a in x], 0), torch.cat([a[1] for a in x], 0)),
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
valid_iter = iter(valid_loader)

test_perm_data = perm_flood_test_data
test_perm_dataset = InMemoryDataset(test_perm_data, processTestIm)
test_perm_loader = torch.utils.data.DataLoader(test_perm_dataset, batch_size=1, shuffle=True, sampler=None,
                  batch_sampler=None, num_workers=0, collate_fn=lambda x: x[0],
                  pin_memory=True, drop_last=False, timeout=0,
                  worker_init_fn=None)
test_perm_iter = iter(test_perm_loader)

[('S1/Ghana_5079_S1.tif', 'QC_v2/Ghana_5079_QC.tif'), ('S1/Ghana_895194_S1.tif', 'QC_v2/Ghana_895194_QC.tif'), ('S1/Ghana_868803_S1.tif', 'QC_v2/Ghana_868803_QC.tif'), ('S1/Ghana_142312_S1.tif', 'QC_v2/Ghana_142312_QC.tif'), ('S1/Ghana_234935_S1.tif', 'QC_v2/Ghana_234935_QC.tif'), ('S1/Ghana_132163_S1.tif', 'QC_v2/Ghana_132163_QC.tif'), ('S1/Ghana_495107_S1.tif', 'QC_v2/Ghana_495107_QC.tif'), ('S1/Ghana_124834_S1.tif', 'QC_v2/Ghana_124834_QC.tif'), ('S1/Ghana_1033830_S1.tif', 'QC_v2/Ghana_1033830_QC.tif'), ('S1/Ghana_277_S1.tif', 'QC_v2/Ghana_277_QC.tif'), ('S1/Ghana_308249_S1.tif', 'QC_v2/Ghana_308249_QC.tif'), ('S1/India_1050276_S1.tif', 'QC_v2/India_1050276_QC.tif'), ('S1/India_764946_S1.tif', 'QC_v2/India_764946_QC.tif'), ('S1/India_118868_S1.tif', 'QC_v2/India_118868_QC.tif'), ('S1/India_533192_S1.tif', 'QC_v2/India_533192_QC.tif'), ('S1/India_180633_S1.tif', 'QC_v2/India_180633_QC.tif'), ('S1/India_244057_S1.tif', 'QC_v2/India_244057_QC.tif'), ('S1/India_691027_S1.tif', 'QC_v2/In

In [ ]:
# UNet definition
import torch
import torch.nn as nn
import torch.nn.functional as TF

class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = TF.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

class UNet_Model(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet_Model, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [ ]:
# Refiner network
class Refiner_Network(nn.Module):
    def __init__(self, seg_model_class, dataset_name, cs_points=None):
        super(Refiner_Network, self).__init__()
        self.cs_points = cs_points

        if dataset_name[:4] == 'sen1':
            n_channels = 2
        elif dataset_name[:4] == 'sen2':
            n_channels = 13
        
        # assumes that seg_model_class have the same input reqs
        self.stage_1  = seg_model_class(n_channels=n_channels, n_classes=2)

        if cs_points:
            self.stage_2  = seg_model_class(n_channels=n_channels+3, n_classes=2)
        else:
            self.stage_2  = seg_model_class(n_channels=n_channels+2, n_classes=2)

        # self.softmax = nn.Softmax(dim=1)

    def forward(self, batch):
        if self.cs_points:
            img, cs_points = batch
        else:
            img = batch

        init_pred = self.stage_1(img)  # assume stage_1 is a regular sem seg network

        # combine input data and initial_prediction
        if self.cs_points:
            combined_input = torch.cat((img, init_pred, cs_points), dim=1)  # combine over the channel dim.
        else:
            combined_input = torch.cat((img, init_pred), dim=1)  # combine over the channel dim.

        refined_pred = self.stage_2(combined_input)

        return refined_pred

In [ ]:
# get models definition
def convertBNtoGN(module, num_groups=16):
  if isinstance(module, torch.nn.modules.batchnorm.BatchNorm2d):
    return nn.GroupNorm(num_groups, module.num_features,
                        eps=module.eps, affine=module.affine)
    if module.affine:
        mod.weight.data = module.weight.data.clone().detach()
        mod.bias.data = module.bias.data.clone().detach()

  for name, child in module.named_children():
      module.add_module(name, convertBNtoGN(child, num_groups=num_groups))

  return module


def get_model(model_name, dataset_name, cs_points=None):
    model_name = model_name.lower() # handle character case error
    dataset_name = dataset_name.lower()

    if dataset_name[:4] == 'sen1':
        n_channels = 2
    elif dataset_name[:4] == 'sen2':
        n_channels = 13
    else:
        print('FATAL: Invalid dataset name "{}"'.format(dataset_name))
        exit()

    if model_name == 'unet':
        model = UNet_Model(n_channels=n_channels, n_classes=2)
    elif model_name == 'fcn':
        from torchvision.models.segmentation import fcn_resnet50
        model = fcn_resnet50(pretrained=False, num_classes=2, pretrained_backbone=False)
        model.backbone.conv1 = nn.Conv2d(n_channels, 64, kernel_size=7, stride=2, padding=3,
                                         bias=False)
        model = convertBNtoGN(model)
    elif model_name == 'refiner':
        model = UNet_Model(n_channels=n_channels, n_classes=2)
        model = Refiner_Network(UNet_Model, dataset_name, cs_points)
    else:
        print('FATAL: Invalid model name "{}"'.format(model_name))
        exit()
    return model

In [ ]:
# net = get_model(MODEL_NAME, 'sen1', crowd_points_path)
net = get_model('refiner', 'sen1', crowd_points_path)

device = torch.device('cuda')
net = net.to(device)

In [ ]:
from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output

def computeIOU(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  intersection = torch.sum(output * target)
  union = torch.sum(target) + torch.sum(output) - intersection
  iou = (intersection + .0000001) / (union + .0000001)
  if iou != iou:
    print("failed, replacing with 0")
    iou = torch.tensor(0).float()
  return iou
  

def computeAccuracy(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  correct = torch.sum(output.eq(target))
  return correct.float() / len(target)
  
def truePositives(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  correct = torch.sum(output * target)
  return correct

def trueNegatives(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  output = (output == 0)
  target = (target == 0)
  correct = torch.sum(output * target)
  return correct

def falsePositives(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  output = (output == 1)
  target = (target == 0)
  correct = torch.sum(output * target)
  return correct

def falseNegatives(output, target):
  output = torch.argmax(output, dim=1).flatten() 
  target = target.flatten()
  no_ignore = target.ne(255).cuda()
  output = output.masked_select(no_ignore)
  target = target.masked_select(no_ignore)
  output = (output == 0)
  target = (target == 1)
  correct = torch.sum(output * target)
  return correct

training_losses = []
training_accuracies = []
training_ious = []
valid_losses = []
valid_accuracies = []
valid_ious = []

In [ ]:
def train_loop(inputs, labels, net, optimizer, scheduler):
  global running_loss
  global running_iou
  global running_count
  global running_accuracy
  # zero the parameter gradients
  optimizer.zero_grad()
  net = net.cuda()
  # forward + backward + optimize
  outputs = net(inputs.cuda())
  loss = criterion(outputs["out"], labels.long().cuda())
  loss.backward()
  optimizer.step()
  scheduler.step()

  running_loss += loss
  running_iou += computeIOU(outputs["out"], labels.cuda())
  running_accuracy += computeAccuracy(outputs["out"], labels.cuda())
  running_count += 1

def validation_loop(validation_data_loader, net):
  global running_loss
  global running_iou
  global running_count
  global running_accuracy
  global max_valid_iou

  global training_losses
  global training_accuracies
  global training_ious
  global valid_losses
  global valid_accuracies
  global valid_ious

  net = net.eval()
  net = net.cuda()
  count = 0
  iou = 0
  loss = 0
  accuracy = 0
  with torch.no_grad():
      for batch in validation_data_loader:
          images = batch['image']
          labels = batch['label']
          if crowd_points_path is None:
            outputs = net(images.cuda())
          else:
            point_img = batch['point_img']
            outputs = net([images.cuda(), point_img.cuda()])
          if type(outputs) is dict:
            outputs = outputs['out']
          valid_loss = criterion(outputs, labels.long().cuda())
          valid_iou = computeIOU(outputs, labels.cuda())
          valid_accuracy = computeAccuracy(outputs, labels.cuda())
          iou += valid_iou
          loss += valid_loss
          accuracy += valid_accuracy
          count += 1

  iou = iou / count
  accuracy = accuracy / count

  if iou > max_valid_iou:
    max_valid_iou = iou
    save_path = os.path.join("checkpoints", "{}_{}_{}.cp".format(RUNNAME, i, iou.item()))
    torch.save(net.state_dict(), save_path)
    print("model saved at", save_path)

  loss = loss / count
  print("Training Loss:", running_loss / running_count)
  print("Training IOU:", running_iou / running_count)
  print("Training Accuracy:", running_accuracy / running_count)
  print("Validation Loss:", loss)
  print("Validation IOU:", iou)
  print("Validation Accuracy:", accuracy)


  training_losses.append(running_loss / running_count)
  training_accuracies.append(running_accuracy / running_count)
  training_ious.append(running_iou / running_count)
  valid_losses.append(loss)
  valid_accuracies.append(accuracy)
  valid_ious.append(iou)

def test_loop(test_data_loader, net):
  net = net.eval()
  # net = net.cuda()
  count = 0
  iou = 0
  loss = 0
  accuracy = 0
  tp = 0
  fp = 0
  tn = 0
  fn = 0
  with torch.no_grad():
      for batch in tqdm(test_data_loader):
          images = batch['image']
          labels = batch['label']

          if crowd_points_path is None:
            outputs = net(images.cuda())
          else:
            point_img = batch['point_img']
            outputs = net([images.cuda(), point_img.cuda()])

          if type(outputs) is dict:
            outputs = outputs['out']
          valid_iou = computeIOU(outputs, labels.cuda())
          iou += valid_iou
          accuracy += computeAccuracy(outputs, labels.cuda())
          tp +=  truePositives(outputs, labels.cuda())
          fp +=  falsePositives(outputs, labels.cuda())
          tn +=  trueNegatives(outputs, labels.cuda())
          fn +=  falseNegatives(outputs, labels.cuda())
          count += 1

  iou = iou / count
  print("Test Mean IOU:", iou)
  print("Total IOU:", (tp.float() / (fn + fp + tp)))
  print("OMISSON:", fn.float() / (fn + tp))
  print("COMMISSON:", fp.float() / (tn + fp))
  print("Test Accuracy:", accuracy / count)

In [ ]:
from tqdm.notebook import tqdm
from IPython.display import clear_output

running_loss = 0
running_iou = 0
running_count = 0
running_accuracy = 0

training_losses = []
training_accuracies = []
training_ious = []
valid_losses = []
valid_accuracies = []
valid_ious = []


def train_epoch(net, optimizer, scheduler, train_iter):
  for (inputs, labels) in tqdm(train_iter):
    train(inputs.cuda(), labels.cuda(), net.cuda(), optimizer, scheduler)
 

def train_validation_loop(net, optimizer, scheduler, train_loader,
                          valid_loader, num_epochs, cur_epoch):
  global running_loss
  global running_iou
  global running_count
  global running_accuracy
  net = net.train()
  running_loss = 0
  running_iou = 0
  running_count = 0
  running_accuracy = 0
  for i in tqdm(range(num_epochs)):
    train_iter = iter(train_loader)
    train_epoch(net, optimizer, scheduler, train_iter)
  clear_output()
  print("Current Epoch:", cur_epoch)
  validation_loop(iter(valid_loader), net)

In [ ]:
import torch 

test_loop(test_all_loader, net)
test_loop(test_flood_loader, net)
test_loop(test_perm_loader, net)

KeyboardInterrupt: ignored

In [ ]:
import os
from IPython.display import display
import matplotlib.pyplot as plt

max_valid_iou = 0
start = 0

epochs = []
training_losses = []
training_accuracies = []
training_ious = []
valid_losses = []
valid_accuracies = []
valid_ious = []

for i in range(start, 1000):
  train_validation_loop(net, optimizer, scheduler, train_loader, valid_loader, 10, i)
  epochs.append(i)
  x = epochs
  plt.plot(x, training_losses, label='training losses')
  plt.plot(x, training_accuracies, 'tab:orange', label='training accuracy')
  plt.plot(x, training_ious, 'tab:purple', label='training iou')
  plt.plot(x, valid_losses, label='valid losses')
  plt.plot(x, valid_accuracies, 'tab:red',label='valid accuracy')
  plt.plot(x, valid_ious, 'tab:green',label='valid iou')
  plt.legend(loc="upper left")

  display(plt.show())

  print("max valid iou:", max_valid_iou)

NameError: ignored